In [2]:
import pandas as pd

data = pd.read_csv("Data/dataset_2048.csv", dtype = str)

In [3]:
labels = list(data['label'])
labels, len(labels)

(['1000000',
  '1000000',
  '1000000',
  '1111111',
  '0000010',
  '1111111',
  '1100001',
  '0000010',
  '0000010',
  '0000000',
  '1100001',
  '1111111',
  '1111111',
  '1000000',
  '0100111',
  '1000000',
  '1100101',
  '1111111',
  '0000010',
  '0000010',
  '0000010',
  '1111111',
  '0000010',
  '1100001',
  '1100001',
  '1100001',
  '0000010',
  '1000000',
  '1111111',
  '0000010',
  '1111111',
  '0000010',
  '1111111',
  '1100001',
  '1111111',
  '0000010',
  '0000010',
  '1111111',
  '1100101',
  '1111111',
  '1100001',
  '1100101',
  '1100001',
  '1100101',
  '1100101',
  '1000000',
  '0000000',
  '1100101',
  '1101111',
  '0000000',
  '1100001',
  '0000000',
  '1111111',
  '1100001',
  '1111111',
  '1000000',
  '1000000',
  '1100001',
  '1000000',
  '1000000',
  '1111111',
  '1000000',
  '1100101',
  '1111101',
  '0000000',
  '1100001',
  '1111111',
  '1111111',
  '0000010',
  '1100001',
  '1000000',
  '1100101',
  '1100101',
  '1100101',
  '1111111',
  '0000010',
  '1100001',

In [4]:
import numpy as np
# count each occurance of 1s and 0s in position 
count = [0,0,0,0,0,0,0]
for i in labels:
    for j in range(7):
        if i[j] == '1':
            count[j] += 1
np.array(count)/100000


array([0.79032, 0.66123, 0.32991, 0.33908, 0.54032, 0.47309, 0.65276])

In [4]:
import torch

model = torch.load("temp/model.pt")
sum(p.numel() for p in model.parameters())

138435367

In [ ]:
import math 
from torch import nn, Tensor

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class Net(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, input_size: int = 64, dropout: float = 0.5):
        super().__init__()
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.embedding = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.feedforward = nn.Sequential(
                                        nn.Flatten(), 
                                        nn.Linear(d_model * input_size, d_model * 16), 
                                        nn.ReLU(), 
                                        nn.Linear(d_model * 16, d_model * 4), 
                                        nn.ReLU(), 
                                        nn.Linear(d_model * 4, 7), 
                                        nn.Sigmoid())

In [3]:
import torch

a = torch.Tensor([1, 2, 3, 4]).to('cuda')